In [1]:
import whisper
from whisper.utils import get_writer
from openai import OpenAI

In [ ]:
model_name = "base"
model = whisper.load_model(model_name, download_root="./models")

audio_path = "./output_audio.aac"
result = model.transcribe(audio_path, language=None, initial_prompt=None)

writer = get_writer("srt", ".")
writer(result, "output.srt")

In [2]:
# Point to the local server
client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")

completion = client.chat.completions.create(
  model="lmstudio-community/Meta-Llama-3.1-8B-Instruct-GGUF",
  messages=[
    {"role": "system", "content": "You are a professional translator. You translate a film script from English to Simplified Chinese. You only reply the translated text."},
    {"role": "user", "content": "Translate this into Simplified Chinese: 'I'm sorry, I can't do that.'"},
  ],
  temperature=0.9,
  max_tokens=100
)

print(completion.choices[0].message.content)

"对不起，我不能那样做。"


In [3]:
import srt
from tqdm import tqdm 

def read_srt_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
    return list(srt.parse(content))

def write_srt_file(file_path, subtitles):
    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(srt.compose(subtitles))
        
def translate_to_chinese(text):
    completion = client.chat.completions.create(
            model="lmstudio-community/Meta-Llama-3.1-8B-Instruct-GGUF",
            messages=[
                {"role": "system", "content": "You are a professional translator. You translate a film script from English to Simplified Chinese. You will only reply the translated text."},
                {"role": "user", "content": f"Translate into Simplified Chinese: {text}"},
            ],
            temperature=0.9,
            max_tokens=100
        )
    return completion.choices[0].message.content

def translate_subtitles(subtitles):
    translated_subtitles = []
    for subtitle in tqdm(subtitles):
        english_text = subtitle.content
        chinese_translation = translate_to_chinese(english_text)
        # print(f"Translating: {english_text} -> {chinese_translation}")
        
        # Combine English and Chinese in the content
        combined_text = f"{english_text}\n{chinese_translation}"
        
        # Create a new subtitle object with combined text
        translated_subtitles.append(srt.Subtitle(index=subtitle.index,
                                                 start=subtitle.start,
                                                 end=subtitle.end,
                                                 content=combined_text))
    return translated_subtitles

In [4]:
input_srt_file = "output.srt"
output_srt_file = "translate.srt"
subtitles = read_srt_file(input_srt_file)
translated_subtitles = translate_subtitles(subtitles)
write_srt_file(output_srt_file, translated_subtitles)
print(f"Translation complete. Check the file {output_srt_file} for the result.")

100%|██████████| 2514/2514 [18:08<00:00,  2.31it/s]

Translation complete. Check the file translate.srt for the result.
